In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/subsam/subsample/50/train_tx.csv
/kaggle/input/subsam/subsample/50/test_tx.csv
/kaggle/input/subsam/subsample/50/test_label.csv
/kaggle/input/subsam/subsample/50/train_label.csv
/kaggle/input/subsam/subsample/10/train_tx.csv
/kaggle/input/subsam/subsample/10/test_tx.csv
/kaggle/input/subsam/subsample/10/test_label.csv
/kaggle/input/subsam/subsample/10/train_label.csv
/kaggle/input/subsam/subsample/100000/train_tx.csv
/kaggle/input/subsam/subsample/100000/test_tx.csv
/kaggle/input/subsam/subsample/100000/test_label.csv
/kaggle/input/subsam/subsample/100000/train_label.csv
/kaggle/input/subsam/subsample/20/train_tx.csv
/kaggle/input/subsam/subsample/20/test_tx.csv
/kaggle/input/subsam/subsample/20/test_label.csv
/kaggle/input/subsam/subsample/20/train_label.csv
/kaggle/input/subsam/subsample/5000/train_tx.csv
/kaggle/input/subsam/subsample/5000/test_tx.csv
/kaggle/input/subsam/subsample/5000/test_label.csv
/kaggle/input/subsam/subsample/5000/train_label.csv
/kaggle/input/su

In [2]:
!pip install pymc_marketing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.9/253.9 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 52.1 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.2/41.2 MB 41.2 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.2
    Uninstalling scipy-1.15.2:
      Successfully uninstalled scipy-1.15.2
  Attempting uninstall: pytensor
    Found existing installation: pytensor 2.27.1
    Uninstalling pytensor-2.27.1:
      Successfully uninstalled pytensor-2.27.1
  Attempting uninstall: pymc
    Found existing installation: pymc 5.20.1
    Uninstalling pymc-5.20.1:
      Successfully uninstalled pymc-5.20.1
ERROR: pip's dependency resolver does not cu

In [3]:
!pip uninstall -y scipy
#!pip install "scipy<1.13"       # e.g. 1.12.0 or 1.11.4
#!pip install --upgrade "numpy<2" # PyMC still needs NumPy 1.x
!pip install --upgrade "numpy>=1.23,<2"      # make sure NumPy is in‑range
!pip install --force-reinstall "scipy>=1.11,<1.13"

Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 46.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 89.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.12.0 which is incompatible.
datasets 3.5.0 requires fsspec[http]<=2024.12.0,>=2023.1.0, but you have fsspec 2025.3.2 which is incompatible.

In [4]:
# ----------------------
# ParetoNBD
#-----------------------
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pymc_marketing.clv.utils import rfm_summary
from pymc_marketing.clv.models import ParetoNBDModel, GammaGammaModel

BASE = Path("/kaggle/input/subsam/subsample/10")
TRAIN_TX  = BASE / "train_tx.csv"
TEST_TX   = BASE / "test_tx.csv"
TEST_LBL  = BASE / "test_label.csv"

def load_csv(path):
    df = pd.read_csv(path)
    df.columns = (
        df.columns
          .str.strip().str.lower()
          .str.replace(r"[^0-9a-z_]", "", regex=True)
          .str.replace(r"_+$", "", regex=True)
    )
    return df

train_tx, test_tx, test_lbl = map(load_csv, [TRAIN_TX, TEST_TX, TEST_LBL])

for col in ["customer_id", "date", "price"]:
    if col not in test_tx.columns:
        raise ValueError(f"Column '{col}' missing in transactions CSV")

train_tx["date"] = pd.to_datetime(train_tx["date"])
test_tx ["date"] = pd.to_datetime(test_tx ["date"])

t0 = test_tx["date"].max().normalize()

hist_train = train_tx.query("date <= @t0")
hist_test  = test_tx .query("date <= @t0")

def build_rfm(df, obs_end):
    return (
        rfm_summary(
            df,
            customer_id_col="customer_id",
            datetime_col="date",
            observation_period_end=obs_end,
            time_unit="D",
        )
        .reset_index()                     
    )

def build_money(df):
    return (
        df.groupby("customer_id", as_index=False)
          .agg(frequency=("price", "count"),
               monetary_value=("price", "mean"))
    )

rfm_train, rfm_test = build_rfm(hist_train, t0), build_rfm(hist_test, t0)
gg_train , gg_test  = build_money(hist_train), build_money(hist_test)

global_avg_spend = train_tx["price"].mean()

pareto = ParetoNBDModel(rfm_train); _ = pareto.fit(method="map")
gg_mod = GammaGammaModel(gg_train.query("frequency > 1")); _ = gg_mod.fit(method="map")

exp_purch = (
    pareto
    .expected_purchases(data=rfm_test, future_t=365)   # <- keyword args
    .to_series()
)

exp_spend_known = gg_mod.expected_customer_spend(
    data=gg_test.query("frequency > 1")
).to_series()

exp_spend = exp_spend_known.reindex(exp_purch.index).fillna(global_avg_spend)

clv_pred = (exp_purch * exp_spend).rename("clv_pred")

y_test_all = test_lbl.set_index("customer_id")["regression_label"]
common_ids = clv_pred.index.intersection(y_test_all.index)

if len(common_ids) == 0:
    print("No overlapping customers between predictions and label file.")
else:
    y_test   = y_test_all.loc[common_ids]
    clv_pred = clv_pred   .loc[common_ids]

    mae  = mean_absolute_error(y_test, clv_pred)
    rmse = mean_squared_error(y_test, clv_pred, squared=False)

    print(f"Prediction start (t0): {t0.date()}  |  Horizon: 365 days")
    print(f"Customers evaluated  : {len(common_ids):,}")
    print(f"Test MAE  : {mae:,.2f}")
    print(f"Test RMSE : {rmse:,.2f}")


Output()

Output()

⚠️  No overlapping customers between predictions and label file.


In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pymc_marketing.clv.utils import rfm_summary
from pymc_marketing.clv.models import BetaGeoModel, GammaGammaModel

BASE = Path("/kaggle/input/subsam/subsample/100")
train_tx = pd.read_csv(BASE / "train_tx.csv"); test_tx = pd.read_csv(BASE / "test_tx.csv"); test_lbl = pd.read_csv(BASE / "test_label.csv")
for df in (train_tx, test_tx): df.columns = df.columns.str.strip().str.lower().str.replace(r"[^0-9a-z_]", "", regex=True).str.replace(r"_+$", "", regex=True); df["date"] = pd.to_datetime(df["date"])
t0 = test_tx["date"].max().normalize()
hist_train = train_tx.query("date <= @t0"); hist_test = test_tx.query("date <= @t0")
rfm_train = rfm_summary(hist_train, "customer_id", "date", t0, time_unit="D").reset_index(); rfm_test = rfm_summary(hist_test, "customer_id", "date", t0, time_unit="D").reset_index()
gg_train = hist_train.groupby("customer_id", as_index=False).agg(frequency=("price","count"), monetary_value=("price","mean")); gg_test = hist_test.groupby("customer_id", as_index=False).agg(frequency=("price","count"), monetary_value=("price","mean"))
global_avg = train_tx["price"].mean()

bg = BetaGeoModel(rfm_train); bg.fit(method="map")
ggm = GammaGammaModel(gg_train.query("frequency>1")); ggm.fit(method="map")

purch = bg.expected_purchases(data=rfm_test, future_t=365).to_series()
spend_known = ggm.expected_customer_spend(data=gg_test.query("frequency>1")).to_series()
spend = spend_known.reindex(purch.index).fillna(global_avg)
clv = (purch*spend).rename("clv_pred")

y = test_lbl.set_index("customer_id")["regression_label"]
ids = clv.index.intersection(y.index)
if ids.any():
    mae = mean_absolute_error(y.loc[ids], clv.loc[ids]); rmse = mean_squared_error(y.loc[ids], clv.loc[ids], squared=False)
    print(mae, rmse)
else:
    print("no overlap")


In [ ]:
import pandas as pd
from pathlib import Path
from sklearn.metrics import mean_absolute_error, mean_squared_error
from pymc_marketing.clv.utils import rfm_summary
from pymc_marketing.clv.models import ModifiedBetaGeoModel, GammaGammaModel

BASE = Path("/kaggle/input/subsam/subsample/100")
train_tx = pd.read_csv(BASE / "train_tx.csv"); test_tx = pd.read_csv(BASE / "test_tx.csv"); test_lbl = pd.read_csv(BASE / "test_label.csv")
for df in (train_tx, test_tx): df.columns = df.columns.str.strip().str.lower().str.replace(r"[^0-9a-z_]", "", regex=True).str.replace(r"_+$", "", regex=True); df["date"] = pd.to_datetime(df["date"])
t0 = test_tx["date"].max().normalize()
hist_train = train_tx.query("date <= @t0"); hist_test = test_tx.query("date <= @t0")
rfm_train = rfm_summary(hist_train, "customer_id", "date", t0, time_unit="D").reset_index(); rfm_test = rfm_summary(hist_test, "customer_id", "date", t0, time_unit="D").reset_index()
gg_train = hist_train.groupby("customer_id", as_index=False).agg(frequency=("price","count"), monetary_value=("price","mean")); gg_test = hist_test.groupby("customer_id", as_index=False).agg(frequency=("price","count"), monetary_value=("price","mean"))
global_avg = train_tx["price"].mean()

mbg = ModifiedBetaGeoModel(rfm_train); mbg.fit(method="map")
ggm = GammaGammaModel(gg_train.query("frequency>1")); ggm.fit(method="map")

purch = mbg.expected_purchases(data=rfm_test, future_t=365).to_series()
spend_known = ggm.expected_customer_spend(data=gg_test.query("frequency>1")).to_series()
spend = spend_known.reindex(purch.index).fillna(global_avg)
clv = (purch*spend).rename("clv_pred")

y = test_lbl.set_index("customer_id")["regression_label"]
ids = clv.index.intersection(y.index)
if ids.any():
    mae = mean_absolute_error(y.loc[ids], clv.loc[ids]); rmse = mean_squared_error(y.loc[ids], clv.loc[ids], squared=False)
    print(mae, rmse)
else:
    print("no overlap")
